In [1]:
#import libraries
%matplotlib inline
from sklearn.linear_model import LogisticRegression
from sklearn import metrics, cross_validation
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
import statsmodels.api as sm
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import *
from collections import Counter
from ggplot import *

In [2]:
data = pd.read_csv("https://raw.githubusercontent.com/paulharris171/cs109b-Movie-Prediction-/master/complete_movie_dataframe.csv?token=AGPPqFHbMgXcP8jy4oQWDNAl4S1virz4ks5ZE2RJwA%3D%3D")

In [3]:
data = data[data['genre_adjusted'] != 'Game-Show']
data = data[data['genre_adjusted'] != 'Film-Noir']
data = data[data['genre_adjusted'] != 'News']
data = data[data['genre_adjusted'] != 'Reality-TV']
data = data[data['genre_adjusted'] != 'Short']

In [10]:
data['genre_adjusted'].unique()

array(['Sci-Fi', 'Action-Comedy', 'Animation', 'Family', 'Western',
       'Adventure', 'History', 'Drama', 'Sport', 'Horror', 'Rom-Com',
       'War', 'Mystery', 'Action', 'Crime', 'Biography', 'Comedy',
       'Musical', 'Documentary', 'Fantasy', 'Thriller', 'Romance', 'Music'], dtype=object)

In [281]:
data.head()

,director_name,budget,genres,actor_1_name,movie_title,plot_keywords,content_rating,genre_adjusted,log_budget,all_keywords
0,James Bobin,170000000.0,"['Adventure', 'Family', 'Fantasy']",Johnny Depp,Alice Through the Looking Glass,"['clock', 'dark fantasy', 'mad hatter', 'queen...",PG,Family,18.951309,"['James Bobin', 'Johnny Depp', 'clock', 'dark'..."
1,Jon M. Chu,90000000.0,"['Action', 'Adventure', 'Comedy', 'Crime', 'My...",Daniel Radcliffe,Now You See Me 2,"['card trick', 'london england', 'magician', '...",PG-13,Mystery,18.315320,"['Jon M. Chu', 'Daniel Radcliffe', 'card', 'tr..."
2,Michael Mann,70000000.0,"['Action', 'Crime', 'Drama', 'Mystery', 'Thril...",Chris Hemsworth,Blackhat,"['chinese', 'computer', 'cybercrime', 'hacker'...",R,Mystery,18.064006,"['Michael Mann', 'Chris Hemsworth', 'chinese',..."
3,Joel Schumacher,70000000.0,"['Drama', 'Musical', 'Romance', 'Thriller']",Gerard Butler,The Phantom of the Opera,"['based on stage musical based on novel', 'dis...",PG-13,Musical,18.064006,"['Joel Schumacher', 'Gerard Butler', 'based', ..."
4,Peter Ramsey,145000000.0,"['Adventure', 'Animation', 'Family', 'Fantasy']",Hugh Jackman,Rise of the Guardians,"['belief', 'box office hit', 'children', 'new ...",PG,Animation,18.792244,"['Peter Ramsey', 'Hugh Jackman', 'belief', 'bo..."


#### Separate the desired features from the genre labels.

In [4]:
#exclude actor name for now, but make a dataframe with features and genre titles 

data_features = data[['all_keywords']]
data_y = data[['genre_adjusted']]
#data_features = pd.get_dummies(data_features, columns=['content_rating'])

#### Split into train/test

In [5]:
X_train, X_test, y_train, y_test = train_test_split(data_features, data_y, test_size=0.33, random_state=41)

# Random Forest x Bag of Words

#### The first approach considered was a random forest fit using a bag of words sourced from plot_keywords. We wondered if keywords alone have significant predictive ability so we checked this. We vectorized our plot_keywords (Below we display a count of the amount of times each keyword appeared in our dataset, we trained a random forest model with the associated genres and then we used this to predict genre from the plot_keywords in the test set.

In [6]:
vectorizer = TfidfVectorizer( max_features = 40000, ngram_range = ( 1, 2 ), 
    sublinear_tf = True )

train_x = vectorizer.fit_transform(X_train['all_keywords'])

test_x = vectorizer.transform(X_test['all_keywords'])

all_x = vectorizer.fit_transform(data_features['all_keywords'])

In [269]:
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

stemmer = PorterStemmer()
stemmed =[]
data2['stemmed'] = data2['all_keywords']


for i in range(len(data2['stemmed'])):
    data2['stemmed'][i] = stem_tokens(data2['all_keywords'][i], stemmer)


C:\Users\Excaliburxiv\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [28]:
#vectorizer = CountVectorizer(analyzer = "word",   
#                             tokenizer = None,    
#                             preprocessor = None, 
#                             stop_words = 'english',   
#                             max_features = 5000) 

#train_data_features = vectorizer.fit_transform(data['plot_keywords'])

#train_data_features = train_data_features.toarray()

In [32]:
#What words do we have?
#vocab = vectorizer.get_feature_names()
#print vocab

In [23]:
# Prints the count and the individual plot_keyword
#dist = np.sum(train_data_features, axis=0)
#for tag, count in zip(vocab, dist):
#    print count, tag

#### Initialize Random Forest Model 

In [350]:
# Random forest model 
forest = RandomForestClassifier(n_estimators = 100) 
forest = forest.fit(train_x, y_train)

C:\Users\Excaliburxiv\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  app.launch_new_instance()


In [351]:
forest.score(test_x,y_test)

0.36617100371747213

In [ ]:
forest.predict(test_x)

#### Using 10 fold CV with Parameter Tuning using Grid Search

In [11]:
# 10-Fold Cross validation with paramter tuning

param_grid = {
                 'n_estimators': [100, 150, 200],
             }

clf = RandomForestClassifier(oob_score = True)
grid_clf = GridSearchCV(clf, param_grid, cv=10)
grid_clf.fit(all_x, data_y['genre_adjusted'])

#print np.mean(cross_val_score(forest, all_x, data_y['genre_adjusted'], cv=5))

GridSearchCV(cv=10, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=True, random_state=None, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [100, 150, 200]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [12]:
print grid_clf.best_params_

{'n_estimators': 200}


In [13]:
print np.mean(cross_val_score(RandomForestClassifier(n_estimators=200), all_x, data_y['genre_adjusted'], cv=10))

0.377764706147


In [14]:
best_model_tree = grid_clf.best_estimator_
best_model_tree

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=True, random_state=None, verbose=0, warm_start=False)

In [19]:
best_model_tree.oob_score_

0.39046839844548986

#### Overall, our random forest approach has a classification accuracy of around 37-39%. 

# Logistic Regression Approach  

In [21]:
log_model = LogisticRegression()
log_model.fit(train_x, y_train)

log_model.predict(train_x)

C:\Users\Excaliburxiv\AppData\Local\Continuum\Anaconda2\lib\site-packages\sklearn\utils\validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array(['Horror', 'Sci-Fi', 'Rom-Com', ..., 'Rom-Com', 'Rom-Com', 'Sci-Fi'], dtype=object)

In [22]:
log_model.score(test_x,y_test)

0.31226765799256506

#### Cross validation for Logistic Regression

In [329]:
predicted = cross_validation.cross_val_predict(LogisticRegression(), all_x, data_y['genre_adjusted'], cv=5)
print metrics.accuracy_score(data_y, predicted)

0.30245398773


#### Logistic Regression returns a classification accuracy of around 30%

In [328]:
log_model = LogisticRegression()
log_model.fit(all_x, data_y)

log_model.score(all_x,data_y)

#log_model.predict(train_x)

C:\Users\Excaliburxiv\AppData\Local\Continuum\Anaconda2\lib\site-packages\sklearn\utils\validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.66012269938650303

#### Discussion of differences between models, and performances: 

#### Random forest - We like the fact that a random forest is non-linear, robust and flexible so we decided to implement it, but because we have so many word features with not many overlaps doesn't surprise us that it does not perform the best (euphemism). 

#### Logistic regression - Simple yet effective classical classification algorithm. Though the data may need some tweaking, the method itself may have its flaws for this project in general. Are the decision boundaries even expectedly linear for our features?? i.e., we expected genres to be distinguishable based upon our features for x_train, but that belief doesn't necessarily make it _true_. Our choice of features was based upon what we _thought_ would work as good features to begin with. 

# Initial Testing

In [ ]:
#Search for The Matrix in TMDb
search = tmdb.Search()
response = search.movie(query='The Matrix')

#Initialize base structure for the posters 
base_url = "http://image.tmdb.org/t/p/w185"
matrix_poster = base_url + response['results'][0]['poster_path']
#Then write code to append the poster path to the end of base_url
print "The link for the movie poster is found below:"
matrix_poster

In [ ]:
this = np.array([[1,1],[1,2]])

In [ ]:
search = tmdb.Search()
response = search.movie(query='The Matrix')
base_url = "http://image.tmdb.org/t/p/w185"
matrix_poster = base_url + response['results'][0]['poster_path']
#Then write code to append the poster path to the end of base_url
print "The link for the movie poster is found below:"
matrix_poster
import urllib
from PIL import Image
urllib.urlretrieve(matrix_poster, "local-filename.jpg")
one = Image.open('local-filename.jpg')

In [ ]:
one2 = one.resize((64, 64), Image.ANTIALIAS)
one2

In [ ]:
um = np.array(one2)

In [ ]:
this = np.expand_dims(um, axis = 0)
this.shape

In [ ]:
yus = np.stack((um,um))

In [ ]:
ruh = np.concatenate((yus,yus))

In [ ]:
np.concatenate((yus,ruh)).shape

In [ ]:
data["movie_title"][1][:-2]

# Implementation

Viewing data genres there are currently 13 genres

In [ ]:
print set(data["genre_adjusted"])
len(set(data["genre_adjusted"]))

The below for loop and function grabs the posters of our dataset and then converts the jpgs into arrays of pixels. However within this small dataset, we have 13 genres but Romance is only one datapoint thus train and test will have different shapes. For this small amount of our end goal, we will remove Romance and only have 12 genres. This is simply our initial deep learning functions.

In [ ]:
#Function grabbing posters
import time
#base url
base_url = "http://image.tmdb.org/t/p/w185"
search = tmdb.Search()
images = 0
genres = []
count = 1
k = 25
switch = False
#Create for loop that cycles through dataframe.movie_title
for i in range(len(data["movie_title"])):
    ####
    #Search for movie.
    movie_title = data["movie_title"][i][:-2]
    response = search.movie(query=movie_title)
    #Create movie poster url.
    print(movie_title)
    if (response['total_results'] != 0):
        if (response['results'][0]['poster_path'] is not None):
            if (data["genre_adjusted"][i] != 'Romance'):
                    #print len(response['results'][0]['poster_path'])
                poster_url = base_url + response['results'][0]['poster_path']
                #Retrieve jpg version from url.
                urllib.urlretrieve(poster_url, "photos/local-file.jpg")
                img = Image.open('photos/local-file.jpg')
                #RGBA check.
                img = img.convert('RGB')
                #Resize.
                img = img.resize((64, 64), Image.ANTIALIAS)
                img = np.array(img)
                img = np.expand_dims(img, axis = 0)
                #Add image to numpy array stack
                if(i == 0):
                    images = img
                else:
                    #print images.shape
                    #print img.shape
                    images = np.concatenate((images, img))

                ## add to genres
                genres = np.append(genres, data["genre_adjusted"][i])
            

    if(count == k):
        k += 25
        time.sleep(3)
        if switch:
            print(count)
            switch = False
        else:
            switch = True
    count += 1

In [ ]:
# Check what comes out 
#print img
#Resize.
img = Image.open('photos/local-file.jpg')
img = img.convert('RGB')
img = img.resize((100, 100), Image.ANTIALIAS)
img = np.array(img)
img = np.expand_dims(img, axis = 0)
img.shape

In [ ]:
img.shape[3] == 3

In [ ]:
genres